There's nothing better to understand the gradient descent algorithm than to code it from scratch. What? you have heard this before ? This time we are switching to gradient descent for multiple linear regression!

Don't hesitate to come back to your Machine Learning course on linear regression to refresh your memory.

Our goal will be to code a multiple linear regression such as :

f
(
x
)
=
β
×
x
+
β
0
=
β
1
×
x
1
+
⋯
+
β
p
×
x
p
+
β
0


Import the following libraries:
Numpy

In [45]:
import numpy as np

Define a Model class that will take two methods:
__init__(self, data), where data will be the dataset containing the training variables. It's the class builder which will allow you to define an attribute
β
0
β
0
​
  (beta_0 in your code) and an attribute
β
β (beta in your code). These attributes represent the coefficients/parameters of the model an we will be initialize them randomly using Numpy (cf: np.random.randn). beta will have to contain a number of random values equal to the number of training variables.
__call__(self, x), a special method that will turn our class into a callable which will return
β
×
x
+
β
0
β×x+β
0
​
  when called.
⚠️ we are now working with matrices and vectors, therefore you will need to use operations that work for these objects ⚠️

In [46]:
class Model():

  def __init__(self, data):
    feature_num = data.shape[1]
    self.beta = np.random.randn(feature_num)
    self.beta_0 = np.random.randn(1)

  def __call__(self, x):
    return self.beta @ x.transpose() + self.beta_0

Import sklearn.datasets

Use the load_diabetes() function to load the diebetes dataset in an object called diabetes.

Print the DESCR attribute of the diabetes object

Save the content of the data attribute in an object named diabetes_data

Save the content of the target attribute in an object named y

In [47]:
from sklearn.datasets import load_diabetes

In [48]:
diabetes = load_diabetes()

In [49]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

:Number of Instances: 442

:Number of Attributes: First 10 columns are numeric predictive values

:Target: Column 11 is a quantitative measure of disease progression one year after baseline

:Attribute Information:
    - age     age in years
    - sex
    - bmi     body mass index
    - bp      average blood pressure
    - s1      tc, total serum cholesterol
    - s2      ldl, low-density lipoproteins
    - s3      hdl, high-density lipoproteins
    - s4      tch, total cholesterol / HDL
    - s5      ltg, possibly log of serum triglycerides level
    - s6      glu, blood sugar level

Note: Each of these 10 feature variables have bee

In [50]:
diabetes_data = diabetes.data
y = diabetes.target

Create an instance of your class Model and display beta_0 and beta

In [51]:
model = Model(diabetes_data)

In [52]:
model.beta_0

array([1.88539277])

In [53]:
model.beta

array([-1.92641425, -0.31268952, -0.45941739, -0.75012968,  1.19094739,
        1.04900894,  0.01669054,  2.01863569, -1.10728734, -0.99813257])

Try doing a first "regression" by running model(diabetes_data[0,:]).

NB: If you don't have the same values as this notebook in output, this is normal since you have initialized your values randomly.

In [54]:
model(diabetes_data[0,:])

array([1.65186172])

This value corresponds to a random prediction of your model. But we don't have any data yet. This time, let's use sklearn to import data.

Visualize y against the predictions using plotly.

In [55]:
import plotly.express as px

In [56]:
fig = px.scatter(
    x=model(diabetes_data),
    y=y,
    title = "Scatter plot of BMI against Diabetes metrics",
    labels ={
        "x":"BMI",
        "y":" Diabetes Metrics"
    })
fig.show()

In [57]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter
 (x=y,
  y=model(diabetes_data),
  mode='markers',
  name = "target vs predictions")
 )

fig.add_trace(go.Scatter
 (x=y,
  y=y,
  mode="lines",
  name = "perfect prediction line")
 )

fig.update_layout(
    title="Target vs Predictions",
    xaxis_title="target",
    yaxis_title="predictions"
    )
fig.show()

Now we need to define a cost function. For a linear regression, we could use MSE :

np.mean((model(input) - y)**2)

Create a function which we'll call mse (for mean square error). This function will take two arguments y_pred & y_true.

In [58]:
def mse(y_pred, y_true):
  return np.mean((y_pred - y_true)**2)

Test your function by inserting model(diabetes_data) & y as arguments.

Calculate the rmse as well

In [59]:
print('MSE: ',mse(model(diabetes_data),y))
print('RMSE: ',np.sqrt(mse(model(diabetes_data),y)))

MSE:  28509.437001208855
RMSE:  168.84737783338198


We're going to need to compute the gradients for our variable model.beta and our constant model.beta_0. To do this, we're going to need to review our derivative formulas. Since we're not here to do math, we're going to give you these formulas.

derive_model_beta = 2/len(y_pred)*(x.transpose() @ (y_pred - y_true))

derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))

Feel free to read this article if you want to know more about the calculation of the derivative: Gradient Descent Derivation

So using the above formulas, code the first function derivative_mse_beta that will take the arguments:

x --> the values for your variable / y_pred --> the values predicted by your model / y_true --> the values of the target variable

In [60]:
def derivative_mse_beta(x, y_pred, y_true):
  derive_model_beta = 2/len(y_pred)*(x.transpose() @ (y_pred - y_true))
  return derive_model_beta

In [61]:
derivative_mse_beta(diabetes_data,model(diabetes_data),y)

array([-1.38530099, -0.3172038 , -4.29889927, -3.24079845, -1.54629902,
       -1.26564593,  2.89052942, -3.14714624, -4.14745982, -2.80689081])

So using the above formulas, now code the derivative_mse_beta_0 function which will take the arguments :

y_pred --> the values predicted by your model / y_true --> the actual values to predict

In [62]:
def derivative_mse_beta_0(y_pred, y_true):
  derive_model_beta_0 = 2/len(y_pred)*(np.sum(y_pred - y_true))
  return derive_model_beta_0

In [63]:
derivative_mse_beta_0(model(diabetes_data),y)

-300.49618278855786

We will try to see if we can minimize our cost function using the two gradients above. To update our variables, we need to subtract their respective gradients. Ex:
param = param - learning_rate * gradient

Set a learning_rate to 0.1

Try to apply your formula on model.beta and model.beta_0.

In [64]:
# Définir le taux d'apprentissage
learning_rate = 0.1

# Afficher les anciennes valeurs des paramètres
print(f"OLD beta (model.beta) = {model.beta}")
print(f"OLD beta_0 (model.beta_0) = {model.beta_0}")

# Mettre à jour les paramètres en utilisant les dérivées partielles de la MSE
model.beta -= learning_rate * derivative_mse_beta(diabetes_data,model(diabetes_data),y)
model.beta_0 -= learning_rate * derivative_mse_beta_0(model(diabetes_data),y)

# Afficher les nouvelles valeurs des paramètres
print(f"NEW beta (model.beta) = {model.beta}")
print(f"NEW beta_0 (model.beta_0) = {model.beta_0}")


OLD beta (model.beta) = [-1.92641425 -0.31268952 -0.45941739 -0.75012968  1.19094739  1.04900894
  0.01669054  2.01863569 -1.10728734 -0.99813257]
OLD beta_0 (model.beta_0) = [1.88539277]
NEW beta (model.beta) = [-1.78788415 -0.28096914 -0.02952746 -0.42604984  1.34557729  1.17557353
 -0.2723624   2.33335031 -0.69254136 -0.71744349]
NEW beta_0 (model.beta_0) = [31.93501105]


We see that the values of the two parameters have changed, let's see how it affected the predictions of the model. Visualize y vs the model's predictions.

In [66]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y,
    y=model(diabetes_data),
    mode='markers',
    name = "target vs predictions"))

fig.add_trace(go.Scatter(
    x=y,
    y=y,
    mode="lines",
    name = "perfect prediction line"))

fig.update_layout(
    title="Target vs Predictions",
    xaxis_title="target",
    yaxis_title="predictions"
    )

fig.show()

We notice the predictions got a little closer to our real data

Recalculate your MSE

In [67]:
print('MSE: ',mse(model(diabetes_data),y))
print('RMSE: ',np.sqrt(mse(model(diabetes_data),y)))

MSE:  20374.793768434512
RMSE:  142.74030183670803


Our MSE has dropped a lot! This is good news but the process of gradient descent is iterative. So you'll have to do it several times before arriving at accurate predictions.

By making a loop, try to repeat the process from above 10,000 times.
Display every 1000 epochs: mse, model.beta & model.beta_0

In [68]:
# Définir le taux d'apprentissage
learning_rate = 0.1

# Définir un nmbre d'epochs
epochs = 10000

In [70]:
model = Model(diabetes_data)

for epoch in range(epochs):
    # Calculer la fonction de perte
    current_loss = mse(model(diabetes_data), y)

    # Mettre à jour les paramètres
    model.beta -= learning_rate * derivative_mse_beta(diabetes_data, model(diabetes_data), y)
    model.beta_0 -= learning_rate * derivative_mse_beta_0(model(diabetes_data), y)

    # Afficher les variables mises à jour à intervalles réguliers
    if epoch % 1000 == 0 or epoch == epochs - 1:
      # epoch % 1000 == 0  => L'opérateur modulo (%) calcule le reste d'une division entière; Dans ce cas, epoch % 1000 calcule le reste de la division de epoch par 1000. Vérification si epoch est un multiple de 1000
      # epoch == epochs - 1 => vérifie si l'itération actuelle est la dernière
      print(f"-------------------- Epoch {epoch} --------------------")
      print(f"Current Loss: {current_loss}")
      print(f"beta = {model.beta}")
      print(f"beta_0 = {model.beta_0}")

-------------------- Epoch 0 --------------------
Current Loss: 29162.520378246616
beta = [-0.03018624  0.40587364  1.17431523 -1.28917526  0.0379771   0.28378157
  0.84952559  1.39464814  0.96248335 -1.55795037]
beta_0 = [30.20830263]
-------------------- Epoch 1000 --------------------
Current Loss: 3408.607487150608
beta = [  48.28500832  -32.56370266  259.8364246   179.84733715   35.97749808
   10.75013396 -147.34730474  134.96838057  230.23156471  127.33463485]
beta_0 = [152.13348416]
-------------------- Epoch 2000 --------------------
Current Loss: 3079.642547220626
beta = [  38.23381477  -93.88832592  368.65769471  243.10790346    6.68161162
  -36.70699265 -185.37956223  144.35437405  312.55114579  142.70371858]
beta_0 = [152.13348416]
-------------------- Epoch 3000 --------------------
Current Loss: 2965.9634157489627
beta = [  24.35074908 -141.92387219  428.49744405  273.88613188  -17.58038469
  -72.38474923 -198.47156353  138.3667479   357.35959482  136.72124226]
beta_0 = [

In [74]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=y,
    y=model(diabetes_data),
    mode='markers',
    name = "target vs predictions"
))

fig.add_trace(go.Scatter(
    x=y,
    y=y,
    mode='lines',
    name = "perfect prediction line"
))

fig.update_layout(
    title="Target vs Predictions",
    xaxis_title="target",
    yaxis_title="predictions"
)

fig.show()